In [62]:
#pip install pyarrow


In [63]:
# Import necessary libraries
import pandas as pd
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import mlflow


In [64]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='/workspaces/mlops_zoombootcamp/02-experiment-tracking/mlruns/1', creation_time=1720343683537, experiment_id='1', last_update_time=1720343683537, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [65]:
#experiment_id = mlflow.create_experiment("New Experiment Name")
#mlflow.set_experiment(experiment_id)

In [66]:
# Function to read and preprocess data
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    df['PULocationID'] = df['PULocationID'].astype('category')
    df['DOLocationID'] = df['DOLocationID'].astype('category')
    return df

In [67]:
# Load and prepare training and validation data
df_train = read_dataframe('data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('data/yellow_tripdata_2023-02.parquet')
target = 'duration'
categorical_columns = ['PULocationID', 'DOLocationID']

<div style="background-color: green; color: white; padding: 10px;">

The purpose of the talk is purely focused on MLOps. I have selected only a few columns in order to keep the model and training process simple.

</div>



In [68]:
df_train['duration'].describe()

count    3.009173e+06
mean     1.420486e+01
std      9.939386e+00
min      1.000000e+00
25%      7.216667e+00
50%      1.155000e+01
75%      1.818333e+01
max      6.000000e+01
Name: duration, dtype: float64

In [69]:
# Vectorize categorical features
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(df_train[categorical_columns].to_dict(orient='records'))
X_val = dv.transform(df_val[categorical_columns].to_dict(orient='records'))
y_train = df_train[target].values
y_val = df_val[target].values

In [70]:
print("Dimensionality (number of columns):", X_val.shape[1])

Dimensionality (number of columns): 2


In [71]:
##LinearRegression

In [72]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_rmse = mean_squared_error(y_val, lr.predict(X_val), squared=False)
print("Linear Regression MSE:", lr_rmse)

Linear Regression MSE: 9.963607595829973


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [73]:
 mlflow.end_run()

In [74]:
from sklearn import linear_model
with mlflow.start_run():
    mlflow.set_tag('developer','Huseyn')
    mlflow.log_param('train-data-path', 'data/yellow_tripdata_2023-01.parquet')
    mlflow.log_param('valid-data-path', 'data/yellow_tripdata_2023-02.parquet')
    alpha=0.01
    ls = linear_model.Lasso(alpha=alpha)
    ls.fit(X_train, y_train)
    ls_rmse = mean_squared_error(y_val, lr.predict(X_val), squared=False)
    mlflow.log_metric('rmse',ls_rmse)
    print("Linear Regression MSE:", ls_rmse)


Linear Regression MSE: 9.963607595829973


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [75]:

alpha_values = [0.01, 0.05, 0.1, 0.5, 1.0]
for alpha in alpha_values:
    with mlflow.start_run():
        mlflow.set_tag('developer', 'Huseyn')
        mlflow.log_param('train-data-path', 'data/yellow_tripdata_2023-01.parquet')
        mlflow.log_param('valid-data-path', 'data/yellow_tripdata_2023-02.parquet')
        mlflow.log_param('alpha', alpha)
        ls = linear_model.Lasso(alpha=alpha)
        ls.fit(X_train, y_train)
        ls_rmse = mean_squared_error(y_val, ls.predict(X_val), squared=False)
        mlflow.log_metric('rmse', ls_rmse)
        print(f"Linear Regression RMSE with alpha={alpha}: {ls_rmse}")


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Linear Regression RMSE with alpha=0.01: 9.96360781849841


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Linear Regression RMSE with alpha=0.05: 9.963608750857988


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Linear Regression RMSE with alpha=0.1: 9.963610009929864


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Linear Regression RMSE with alpha=0.5: 9.963623827395615
Linear Regression RMSE with alpha=1.0: 9.96365046141962


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [76]:
import os
import joblib
model_folder = 'models'
model_path = os.path.join(model_folder, 'lasso_model.pkl')
if not os.path.exists(model_folder):
    os.makedirs(model_folder)
joblib.dump(ls, model_path)
print(f"Model saved successfully in {model_path}!")


Model saved successfully in models/lasso_model.pkl!


In [77]:
mlflow.log_artifact(local_path="models/lasso_model.pkl", artifact_path='model')

In [78]:
# Save the models and DictVectorizer
with open('models/lasso_model.pkl', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

 #Xgboost model

In [79]:
import xgboost as xgb
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials


In [80]:
import mlflow
import xgboost as xgb
from sklearn.metrics import mean_squared_error

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        model = xgb.XGBRegressor(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            learning_rate=params['learning_rate'],
            subsample=params['subsample'],
            gamma=params['gamma'],
            colsample_bytree=params['colsample_bytree']
        )
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        rmse = mean_squared_error(y_val, preds, squared=False)
        mlflow.log_metric("rmse", rmse)
    return {'loss': -rmse, 'status': STATUS_OK}


In [81]:
space = {
    'max_depth': hp.choice('max_depth', range(1, 3)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.02),
    'n_estimators': hp.choice('n_estimators', range(100, 101)),
    'subsample': hp.uniform('subsample', 0.7, 0.8),
    'gamma': hp.uniform('gamma', 0.0, 0.1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
}


In [82]:
mlflow.end_run()

In [83]:
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=5,
            trials=trials)
print("Best hyperparameters:", best)


 20%|██        | 1/5 [00:08<00:33,  8.28s/trial, best loss: -9.054027860453663]

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



 40%|████      | 2/5 [00:16<00:24,  8.25s/trial, best loss: -9.141173531077797]

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



 60%|██████    | 3/5 [00:22<00:14,  7.43s/trial, best loss: -9.783727250198924]

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



 80%|████████  | 4/5 [00:31<00:07,  7.84s/trial, best loss: -9.783727250198924]

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



100%|██████████| 5/5 [00:39<00:00,  7.89s/trial, best loss: -9.783727250198924]
Best hyperparameters: {'colsample_bytree': 0.30250167658822336, 'gamma': 0.05751433712431652, 'learning_rate': 0.015099626762582682, 'max_depth': 0, 'n_estimators': 0, 'subsample': 0.7064508683090851}


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [84]:
## Trainin model with the best hyperparamters 

In [85]:
import xgboost as xgb

# Preparing training and validation datasets
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)


In [86]:
hyperparams ={'colsample_bytree':0.9320267530659203,
            'gamma':0.0976458782129943,
            'learning_rate':0.019905481508947585,
            'max_depth':2,
            'n_estimators':100,
            'subsample':0.7513506613595762}

In [87]:


with mlflow.start_run():
    mlflow.set_tag("Developer", "Huseyn")
    mlflow.log_params(hyperparams)
    booster=xgb.train(
                params=hyperparams,
                dtrain=train,
                num_boost_round=50,
                early_stopping_rounds=20,
                evals=[(valid, "validation")])
    y_pred=booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv,f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
   

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [09:25:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.05957
[1]	validation-rmse:10.02447
[2]	validation-rmse:10.01667
[3]	validation-rmse:9.98292
[4]	validation-rmse:9.97636
[5]	validation-rmse:9.96900
[6]	validation-rmse:9.96278
[7]	validation-rmse:9.95681
[8]	validation-rmse:9.95108
[9]	validation-rmse:9.91861
[10]	validation-rmse:9.91313
[11]	validation-rmse:9.88181
[12]	validation-rmse:9.87655
[13]	validation-rmse:9.87150
[14]	validation-rmse:9.86671
[15]	validation-rmse:9.83662
[16]	validation-rmse:9.80764
[17]	validation-rmse:9.80295
[18]	validation-rmse:9.77503
[19]	validation-rmse:9.74812
[20]	validation-rmse:9.74365
[21]	validation-rmse:9.71768
[22]	validation-rmse:9.69267
[23]	validation-rmse:9.66852
[24]	validation-rmse:9.66245
[25]	validation-rmse:9.65823
[26]	validation-rmse:9.63501
[27]	validation-rmse:9.63100
[28]	validation-rmse:9.62530
[29]	validation-rmse:9.60301
[30]	validation-rmse:9.58152
[31]	validation-rmse:9.56081
[32]	validation-rmse:9.54087
[33]	validation-rmse:9.52161
[34]	validation-rmse:

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [09:25:39] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
/home/codespace/.python/current/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an edita

In [88]:
##Autolog mlfow 

In [89]:

mlflow.xgboost.autolog()
booster=xgb.train(
            params=hyperparams,
            dtrain=train,
            num_boost_round=100,
            early_stopping_rounds=50,
            evals=[(valid, "validation")])
y_pred=booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred, squared=False)
mlflow.log_metric("rmse", rmse)

2024/07/07 09:25:42 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.


2024/07/07 09:25:42 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3ba791e1ac744a76944a782f180031c3', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [09:25:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.05957
[1]	validation-rmse:10.02447
[2]	validation-rmse:10.01667
[3]	validation-rmse:9.98292
[4]	validation-rmse:9.97636
[5]	validation-rmse:9.96900
[6]	validation-rmse:9.96278
[7]	validation-rmse:9.95681
[8]	validation-rmse:9.95108
[9]	validation-rmse:9.91861
[10]	validation-rmse:9.91313
[11]	validation-rmse:9.88181
[12]	validation-rmse:9.87655
[13]	validation-rmse:9.87150
[14]	validation-rmse:9.86671
[15]	validation-rmse:9.83662
[16]	validation-rmse:9.80764
[17]	validation-rmse:9.80295
[18]	validation-rmse:9.77503
[19]	validation-rmse:9.74812
[20]	validation-rmse:9.74365
[21]	validation-rmse:9.71768
[22]	validation-rmse:9.69267
[23]	validation-rmse:9.66852
[24]	validation-rmse:9.66245
[25]	validation-rmse:9.65823
[26]	validation-rmse:9.63501
[27]	validation-rmse:9.63100
[28]	validation-rmse:9.62530
[29]	validation-rmse:9.60301
[30]	validation-rmse:9.58152
[31]	validation-rmse:9.56081
[32]	validation-rmse:9.54087
[33]	validation-rmse:9.52161
[34]	validation-rmse:

2024/07/07 09:28:14 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/07/07 09:28:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [09:28:14] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/07/07 09:28:15 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/.python/current/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid usi

In [90]:
import mlflow
logged_model = 'runs:/xxxxxx/models_mlflow'
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model

MlflowException: Run with id=f057b62d35924722b03678f9001d984e not found

In [ ]:
loaded_model = mlflow.pyfunc.load_model(logged_model)
import mlflow.xgboost
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [ ]:
xgboost_model

In [ ]:
xgboost_model.predict(valid)

array([12.872271, 26.821747, 12.919162, ..., 12.919162, 13.479717,
       12.919162], dtype=float32)

### Deep Learning

In [ ]:

class TaxiTripModel(nn.Module):
    def __init__(self):
        super(TaxiTripModel, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        return self.output(x)

In [ ]:
# Convert data to PyTorch tensors and create DataLoaders
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_data = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

In [ ]:
# Train the PyTorch model
model = TaxiTripModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

In [ ]:
def train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            preds = model(X_batch)
            loss = loss_fn(preds, y_batch.unsqueeze(1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        model.eval()
        val_loss = sum(loss_fn(model(X_val), y_val.unsqueeze(1)) for X_val, y_val in val_loader) / len(val_loader)
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}')

In [ ]:
train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=10)

# Save the trained PyTorch model and DictVectorizer
with open('models/TaxiTripModel.bin', 'wb') as f_out:
    pickle.dump((dv, model), f_out)

## Deploying model

In [ ]:
from mlflow.tracking import MlflowClient
mlflow_tracking_uri="sqlite:///mlflow.db"
client= MlflowClient(tracking_uri=mlflow_tracking_uri)

In [ ]:
import mlflow

mlflow.search_experiments ()

[<Experiment: artifact_location='/workspaces/mlopsProduction/02-experiment-tracking/mlruns/1', creation_time=1716364782674, experiment_id='1', last_update_time=1716364782674, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716361841500, experiment_id='0', last_update_time=1716361841500, lifecycle_stage='active', name='Default', tags={}>]

In [ ]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results= 5,
)

In [ ]:
for run in runs:
    if 'rmse' in run.data.metrics:
        print(f"Run ID is {run.info.run_id}, RMSE: {run.data.metrics['rmse']:.4f}")
    else:
        print(f"Run ID is {run.info.run_id}, RMSE: Not available")


Run ID is f057b62d35924722b03678f9001d984e, RMSE: 8.9794
Run ID is b151d8b09fb54022873cb16b0f9102d0, RMSE: 8.9794
Run ID is 800c1181925a4115896d68daddc19086, RMSE: 8.9794
Run ID is 972e2ddbbf6b4f4bbf3b912844899036, RMSE: Not available
Run ID is 1d050bfba2d84cab915482be291d471f, RMSE: Not available


In [ ]:
import mlflow

In [ ]:
run_id="f057b62d35924722b03678f9001d984e"
model_uri=f"runs:/{run_id}/model"

In [ ]:
model_uri

'runs:/f057b62d35924722b03678f9001d984e/model'

In [ ]:
mlflow.set_tracking_uri(mlflow_tracking_uri)

In [ ]:
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-experiment")

Successfully registered model 'nyc-taxi-experiment'.
Created version '1' of model 'nyc-taxi-experiment'.


<ModelVersion: aliases=[], creation_timestamp=1716930879755, current_stage='None', description=None, last_updated_timestamp=1716930879755, name='nyc-taxi-experiment', run_id='f057b62d35924722b03678f9001d984e', run_link=None, source='/workspaces/mlopsProduction/02-experiment-tracking/mlruns/1/f057b62d35924722b03678f9001d984e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [ ]:
latest_version =client.get_latest_versions(name= "nyc-taxi-experiment")

/tmp/ipykernel_10974/4195685249.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_version =client.get_latest_versions(name= "nyc-taxi-experiment")


In [ ]:
for version in latest_version:
    print(f'version: {version.version} , stage : {version.current_stage}')

version: 1 , stage : None


In [ ]:
client.transition_model_version_stage(name="nyc-taxi-experiment",
 version=1,
 stage="Staging")

/tmp/ipykernel_10974/3101054215.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(name="nyc-taxi-experiment",


<ModelVersion: aliases=[], creation_timestamp=1716930879755, current_stage='Staging', description=None, last_updated_timestamp=1716931408409, name='nyc-taxi-experiment', run_id='f057b62d35924722b03678f9001d984e', run_link=None, source='/workspaces/mlopsProduction/02-experiment-tracking/mlruns/1/f057b62d35924722b03678f9001d984e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [ ]:
client = mlflow.tracking.MlflowClient()


In [ ]:
client.set_model_version_tag(
    name="nyc-taxi-experiment",
    version=1,
    key="validation_status",
    value="approved"
)

In [ ]:
client.update_registered_model
(name = model_name,
version =4, 
description = f"The model version {version} was"
)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3856153427.py, line 2)